# Create supplement price data of ES installers 
By incorporating TTS price info 

In [8]:
import pandas as pd

## Input:  the es_tts_matchingtable excel file 

In [9]:
tts_es_matchingtble='../0_data/es_tts_matchingtable.xlsx'

In [10]:
tts_es_matching=pd.read_excel(tts_es_matchingtble,sheet_name='v1')

In [11]:
tts_es_matching.head()

,ES_name,score,pass,reviewscount,es_installer_id,ES_areaofservice,ES_link,TTS_names
0,Sunnova Energy Corporation,80,1,NaN,20454,NaN,https://www.energysage.com/supplier/20454/sunn...,sunnova
1,Sunlight Solar Energy,98,1,61.0,566,"CO,CT,MA,OR",https://www.energysage.com/supplier/566/sunlig...,Sunlight Solar Energy
2,"Green Power Energy, LLC",86,1,58.0,20054,"NJ,PA",https://www.energysage.com/supplier/20054/gree...,Green Power Energy
3,LuxLight Solar Energy,84,1,58.0,20348,CA,https://www.energysage.com/supplier/20348/luxl...,Luxlight Solar/ Applied Electric
4,Mass Renewables Inc.,83,1,52.0,6507,MA,https://www.energysage.com/supplier/6507/mass-...,Mass Renewables


## Inpute: TTS original data 

In [12]:
tts_price_url='../0_data/TTS_v2018.dta'

In [13]:
tts_price=pd.read_stata(tts_price_url)

In [14]:
tts_price.head()

,systemidtrackingthesun,installationdate,systemsize,totalinstalledprice,salestaxcost,customersegment,zipcode,city,county,state,utilityserviceterritory,installername,installationdate2,year,month,ones,marketsize,unit_price
0,CA_SMUD_15086,10/7/2016,7.848,NaN,-9999.0,RES,95831.0,"""Sacramento",,,,"""",20734.0,2016.0,10.0,1.0,2.0,NaN
1,CA_SMUD_15535,10/19/2016,7.680,NaN,-9999.0,RES,95829.0,"""Sacramento",,,,"""",20746.0,2016.0,10.0,1.0,2.0,NaN
2,CA_SMUD_15854,11/3/2016,7.848,NaN,-9999.0,RES,95826.0,"""Sacramento",,,,"""",20761.0,2016.0,11.0,1.0,1.0,NaN
3,CA_SMUD_15825,1/4/2017,10.350,NaN,-9999.0,RES,95630.0,"""Folsom",,,,"""",20823.0,2017.0,1.0,1.0,1.0,NaN
4,,,NaN,NaN,NaN,,NaN,,,,,,NaN,NaN,NaN,1.0,9.0,NaN


In [15]:
tts_price=tts_price[['installername','year','month','unit_price']]

In [16]:
tts_price=tts_price.dropna()

In [17]:
tts_price.head()

,installername,year,month,unit_price
14,wayne larue,2010.0,1.0,2.349328
15,liberty solar solutions,2010.0,4.0,7.254167
16,liberty solar solutions,2010.0,4.0,8.874493
17,william ball,2010.0,4.0,6.231426
18,liberty solar solutions,2010.0,4.0,8.496838


In [18]:
tts_es_matching['TTS_names']=tts_es_matching['TTS_names'].str.lower()
#df.applymap(lambda s:s.lower() if type(s) == str else s)

In [19]:
tts_es_matching.head()

,ES_name,score,pass,reviewscount,es_installer_id,ES_areaofservice,ES_link,TTS_names
0,Sunnova Energy Corporation,80,1,NaN,20454,NaN,https://www.energysage.com/supplier/20454/sunn...,sunnova
1,Sunlight Solar Energy,98,1,61.0,566,"CO,CT,MA,OR",https://www.energysage.com/supplier/566/sunlig...,sunlight solar energy
2,"Green Power Energy, LLC",86,1,58.0,20054,"NJ,PA",https://www.energysage.com/supplier/20054/gree...,green power energy
3,LuxLight Solar Energy,84,1,58.0,20348,CA,https://www.energysage.com/supplier/20348/luxl...,luxlight solar/ applied electric
4,Mass Renewables Inc.,83,1,52.0,6507,MA,https://www.energysage.com/supplier/6507/mass-...,mass renewables


In [20]:
tts_es_matching[:3]

,ES_name,score,pass,reviewscount,es_installer_id,ES_areaofservice,ES_link,TTS_names
0,Sunnova Energy Corporation,80,1,NaN,20454,NaN,https://www.energysage.com/supplier/20454/sunn...,sunnova
1,Sunlight Solar Energy,98,1,61.0,566,"CO,CT,MA,OR",https://www.energysage.com/supplier/566/sunlig...,sunlight solar energy
2,"Green Power Energy, LLC",86,1,58.0,20054,"NJ,PA",https://www.energysage.com/supplier/20054/gree...,green power energy


## iterate over tts data to create unit price data

We find the problem is that data is largely incomplete. Monthly level is unachievable. Yearly will be incomplete too. The best thing we can do is to create a table of yearly unit price. And merge the most recent one to the final data 

In [21]:
tts_price_collect=[]
for index,row in tts_es_matching.iterrows():
    #print(row['TTS_names'])
    temp=tts_price[tts_price['installername']==row['TTS_names']]
    name=row['TTS_names']
    #print(temp)
    years=set(temp.year)
    for year in years:
        unit_price_avg=temp['unit_price'][temp['year']==year].mean()
        #print(name, year,unit_price_avg)
        tts_price_collect=tts_price_collect+[[name, year,unit_price_avg]]
tts_price_collect_df=pd.DataFrame(data=tts_price_collect,columns=['tts_name','year','unit_price_avg'])   
tts_price_collect_df.head()

,tts_name,year,unit_price_avg
0,sunnova,2016.0,0.000157
1,sunnova,2017.0,11.904764
2,sunnova,2015.0,1.996771
3,sunlight solar energy,2016.0,3.877920
4,sunlight solar energy,2017.0,3.429872


In [22]:
tts_es_matching.head()

,ES_name,score,pass,reviewscount,es_installer_id,ES_areaofservice,ES_link,TTS_names
0,Sunnova Energy Corporation,80,1,NaN,20454,NaN,https://www.energysage.com/supplier/20454/sunn...,sunnova
1,Sunlight Solar Energy,98,1,61.0,566,"CO,CT,MA,OR",https://www.energysage.com/supplier/566/sunlig...,sunlight solar energy
2,"Green Power Energy, LLC",86,1,58.0,20054,"NJ,PA",https://www.energysage.com/supplier/20054/gree...,green power energy
3,LuxLight Solar Energy,84,1,58.0,20348,CA,https://www.energysage.com/supplier/20348/luxl...,luxlight solar/ applied electric
4,Mass Renewables Inc.,83,1,52.0,6507,MA,https://www.energysage.com/supplier/6507/mass-...,mass renewables


In [23]:
test=pd.merge(left=tts_es_matching,right=tts_price_collect_df,left_on='TTS_names',right_on='tts_name',how='outer')

In [24]:
test.describe()

,score,pass,reviewscount,es_installer_id,year,unit_price_avg
count,902.000000,902.0,899.000000,902.000000,874.000000,874.000000
mean,88.658537,1.0,19.408231,17385.242794,2013.189931,4.949861
std,9.134513,0.0,15.112981,7572.846771,3.259398,1.690020
min,56.000000,1.0,1.000000,424.000000,1999.000000,0.000157
25%,83.000000,1.0,6.000000,20101.000000,2011.000000,3.736164
50%,91.000000,1.0,18.000000,20916.000000,2014.000000,4.464357
75%,96.000000,1.0,29.000000,21576.000000,2016.000000,5.922386
max,100.000000,1.0,61.000000,22997.000000,2017.000000,11.904764


In [25]:
test.tail()

,ES_name,score,pass,reviewscount,es_installer_id,ES_areaofservice,ES_link,TTS_names,tts_name,year,unit_price_avg
897,"T and M Solar, Inc dba America's Choice Contra...",61,1,23.0,21170,CA,https://www.energysage.com/supplier/21170/t-an...,t & m solar,t & m solar,2011.0,5.333333
898,"T and M Solar, Inc dba America's Choice Contra...",61,1,23.0,21170,CA,https://www.energysage.com/supplier/21170/t-an...,t & m solar,t & m solar,2012.0,4.672706
899,"T and M Solar, Inc dba America's Choice Contra...",61,1,23.0,21170,CA,https://www.energysage.com/supplier/21170/t-an...,t & m solar,t & m solar,2013.0,3.612500
900,"T and M Solar, Inc dba America's Choice Contra...",61,1,23.0,21170,CA,https://www.energysage.com/supplier/21170/t-an...,t & m solar,t & m solar,2014.0,4.265014
901,"T and M Solar, Inc dba America's Choice Contra...",61,1,23.0,21170,CA,https://www.energysage.com/supplier/21170/t-an...,t & m solar,t & m solar,2015.0,3.751662


## output: export the matched price

In [26]:
test.to_csv('../2_pipeline/es_tts_unitprices.csv')

## continue merge price info to panel 

In [28]:
price_info=pd.read_csv('../2_pipeline/es_tts_unitprices.csv')

In [29]:
price_info=price_info[['es_installer_id','year','unit_price_avg']]

In [23]:
price_info=price_info.drop_duplicates()

In [30]:
parameters='90_100_2twostep'

## input: es panel data from step 4 

In [31]:
es_panel=pd.read_csv('../2_pipeline/es_panel_step4'+parameters+'.csv')

In [32]:
es_panel['year']=es_panel['year_month_count']/12+2013
es_panel['year']=es_panel.year.astype(int)

## Input: es_marketlevel data from step 3 

In [36]:
parameters

'90_100_2two_step'

In [37]:
es_merged=pd.read_csv('../2_pipeline/es_marketlevel'+parameters+'.csv')

## find competitor and competitor prices 

### helper data list 'es_market_ids' 

In [38]:
es_market_ids=es_merged[['installer_id','year_month_count','market']]

In [39]:
es_market_ids.head()

,installer_id,year_month_count,market
0,21283,35.0,0.0
1,21283,36.0,0.0
2,21283,37.0,0.0
3,21283,38.0,0.0
4,21283,39.0,0.0


### create year variable 
we have to use year to be congruent with the price info 

In [40]:
es_market_ids['year']=es_market_ids.year_month_count/12+2013

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
es_market_ids['year']=es_market_ids['year'].astype(int)
es_market_ids=es_market_ids[['installer_id','market','year']].drop_duplicates()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
es_market_ids.describe()

,installer_id,market,year
count,1068.000000,1068.000000,1068.000000
mean,18571.821161,17.418539,2016.349251
std,6391.070129,13.397041,1.368658
min,108.000000,-1.000000,2013.000000
25%,20314.000000,4.000000,2016.000000
50%,20769.000000,17.000000,2017.000000
75%,21534.000000,33.000000,2017.000000
max,23027.000000,36.000000,2018.000000


### create the data frame to accept variables at installer-market-year level 

In [43]:
tts_priceinfo_for_es=es_merged[['installer_id','market','year_month_count']]

In [44]:
tts_priceinfo_for_es.head()

,installer_id,market,year_month_count
0,21283,0.0,35.0
1,21283,0.0,36.0
2,21283,0.0,37.0
3,21283,0.0,38.0
4,21283,0.0,39.0


In [45]:
tts_priceinfo_for_es['year']=tts_priceinfo_for_es['year_month_count']/12+2013
tts_priceinfo_for_es['year']=tts_priceinfo_for_es['year'].astype(int)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [46]:
tts_priceinfo_for_es.head()

,installer_id,market,year_month_count,year
0,21283,0.0,35.0,2015
1,21283,0.0,36.0,2016
2,21283,0.0,37.0,2016
3,21283,0.0,38.0,2016
4,21283,0.0,39.0,2016


In [47]:
price_info.tail(10)

,es_installer_id,year,unit_price_avg
892,21983,2014.0,3.845287
893,21442,2010.0,8.000000
894,21170,2016.0,4.547740
895,21170,2017.0,4.310848
896,21170,2010.0,6.756757
897,21170,2011.0,5.333333
898,21170,2012.0,4.672706
899,21170,2013.0,3.612500
900,21170,2014.0,4.265014
901,21170,2015.0,3.751662


# Find the own and competitor prices 

In [48]:
tts_priceinfo_for_es.describe()

,installer_id,market,year_month_count,year
count,8113.000000,8113.000000,8113.000000,8113.000000
mean,17971.731912,17.784667,43.258228,2016.154813
std,6707.006717,13.486967,14.543701,1.260144
min,108.000000,-1.000000,1.000000,2013.000000
25%,20131.000000,4.000000,34.000000,2015.000000
50%,20627.000000,17.000000,46.000000,2016.000000
75%,21350.000000,33.000000,55.000000,2017.000000
max,23027.000000,36.000000,64.000000,2018.000000


In [49]:
collect_list=[]
for index, row in tts_priceinfo_for_es.iterrows():
    installer=row['installer_id']
    market=row['market']
    year=row['year']
    own_price=price_info[(price_info['es_installer_id']==installer)&(price_info['year']==year)]['unit_price_avg'].mean()
    slice_df=tts_priceinfo_for_es[tts_priceinfo_for_es['market']==market]
    others=set(slice_df.installer_id)-set({installer})
    other_prices=price_info[(price_info['es_installer_id'].isin(others))&(price_info['year']==year)]['unit_price_avg'].mean()
    collect_list=collect_list+[[installer,market,year,own_price,other_prices]]
collect_list_df=pd.DataFrame(data=collect_list,columns=['installer_id','market','year','own_price','other_prices'])

In [50]:
collect_list_df.describe()

,installer_id,market,year,own_price,other_prices
count,8113.000000,8113.000000,8113.000000,2556.000000,6010.000000
mean,17971.731912,17.784667,2016.154813,3.890053,3.849475
std,6707.006717,13.486967,1.260144,0.855896,0.452125
min,108.000000,-1.000000,2013.000000,1.581735,2.389912
25%,20131.000000,4.000000,2015.000000,3.452381,3.602776
50%,20627.000000,17.000000,2016.000000,3.750083,3.812642
75%,21350.000000,33.000000,2017.000000,4.250046,4.079409
max,23027.000000,36.000000,2018.000000,10.156904,5.362645


In [51]:
collect_list_df.drop_duplicates().describe()

,installer_id,market,year,own_price,other_prices
count,1068.000000,1068.000000,1068.000000,293.000000,697.000000
mean,18571.821161,17.418539,2016.349251,3.900650,3.867437
std,6391.070129,13.397041,1.368658,0.843114,0.449473
min,108.000000,-1.000000,2013.000000,1.581735,2.389912
25%,20314.000000,4.000000,2016.000000,3.430384,3.626963
50%,20769.000000,17.000000,2017.000000,3.791826,3.812642
75%,21534.000000,33.000000,2017.000000,4.254330,4.081263
max,23027.000000,36.000000,2018.000000,10.156904,5.362645


In [52]:
parameters

'90_100_2two_step'

## Output 2_pipeline/'price_for_es_from_'+parameters+'.csv'

In [53]:
collect_list_df.drop_duplicates().to_csv('../2_pipeline/price_for_es_from_'+parameters+'.csv')